In [7]:
!pip install graphene

In [8]:
from rdflib import Graph, Namespace
import graphene
from pyvis.network import Network

ModuleNotFoundError: No module named 'graphene'

In [ ]:
# Load the RDF graph
g = Graph()
try:
    # Replace 'iprok.owl' with the actual path to your RDF file
    g.parse("../../Output/ifc_bimonto.owl", format="xml")  # Adjust format if needed (e.g., 'turtle', 'n3')
    print("RDF data loaded successfully.")
except Exception as e:
    print(f"Error loading RDF file: {e}")
    exit()

In [ ]:
# Function to extract the local part of a URI
def extract_local_name(uri):
    return uri.split('/')[-1].split('#')[-1]

In [ ]:
# Graphene GraphQL Schema
class RDFNode(graphene.ObjectType):
    subject = graphene.String()
    predicate = graphene.String()
    obj = graphene.String()

In [ ]:
!pip show pyvis

In [ ]:
# Graphene GraphQL Query (Dynamic)
class Query(graphene.ObjectType):
    all_triples = graphene.List(
        RDFNode,
        predicate_filter=graphene.String(),
        subject_filter=graphene.String(),
        object_filter=graphene.String()
    )
    
    def resolve_all_triples(self, info, predicate_filter=None, subject_filter=None, object_filter=None):
        triples = []
        for subj, pred, obj in g:
            local_subj = extract_local_name(str(subj))
            local_pred = extract_local_name(str(pred))
            local_obj = extract_local_name(str(obj))
            # Apply AND filtering logic
            if predicate_filter and local_pred != predicate_filter:
                continue
            if subject_filter and local_subj != subject_filter:
                continue
            triples.append(
                RDFNode(
                    subject=local_subj,
                    predicate=local_pred,
                    obj=local_obj
                )
            )
        return triples

schema = graphene.Schema(query=Query)

In [ ]:
# Example GraphQL Query with Filter
query = '''
{
  allTriples{
    subject
    predicate
    obj
  }
}
'''

# Execute the query
result = schema.execute(query)

# Print formatted output
import json
print(json.dumps(result.data, indent=4))

In [ ]:
# Define the predicates and data types to filter out
predicates_to_filter = {
    "type",  # rdf:type
}

predicates_to_filter = {
    "type",  # rdf:type
}

data_types_to_filter = {
    "boolean",
    "string",
    "integer",
    "decimal"
}

In [ ]:
# Create a PyVis Network object with better layout settings
net = Network(height='600px', width='100%', directed=False, notebook=True, neighborhood_highlight=False, select_menu=False, 
              filter_menu=False, bgcolor='#ffffff', font_color=False, layout=None, heading='', cdn_resources='remote')

# Define colors for subjects and objects
subject_color = "#FF5733"  # Orange-red
object_color = "#33FF57"   # Green

# Add nodes and edges, filtering out specified predicates and data types
for triple in result.data['allTriples']:
    subject = triple['subject']
    predicate = triple['predicate']
    obj = triple['obj']
    
    # Skip triples with specified predicates or schema-related data types
    if predicate in predicates_to_filter or obj in data_types_to_filter:
        continue
    
    # Add nodes with specific colors
    net.add_node(subject, label=subject, size=20, color=subject_color)
    net.add_node(obj, label=obj, size=20, color=object_color)
    
    # Add edges with predicate as title and label
    net.add_edge(subject, obj, label=predicate)

# Configure physics settings to improve layout
net.force_atlas_2based(gravity=0.1, central_gravity=0.1)
net.repulsion(node_distance=200)

# Show the interactive graph in the notebook
net.show("rdf_graph2.html")